# Przykład ekstrakcji informacji w spaCy

## Przygotowanie

In [1]:
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install libmorfeusz2
!sudo apt install python3-morfeusz2
!wget http://download.sgjp.pl/morfeusz/20210221/Linux/18.04/64/morfeusz2-1.9.17-cp37-cp37m-linux_x86_64.whl -O morfeusz2-1.9.17-cp37-cp37m-linux_x86_64.whl
!python3 -m pip install --upgrade --force-reinstall morfeusz2-1.9.17-cp37-cp37m-linux_x86_64.whl

--2021-01-25 12:00:35--  http://download.sgjp.pl/apt/sgjp.gpg.key
Resolving download.sgjp.pl (download.sgjp.pl)... 193.0.67.154
Connecting to download.sgjp.pl (download.sgjp.pl)|193.0.67.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1029 (1.0K) [application/pgp-keys]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.00K  --.-KB/s    in 0s      

2021-01-25 12:00:36 (154 MB/s) - written to stdout [1029/1029]

OK
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hi

In [2]:
! wget -P zasoby http://mozart.ipipan.waw.pl/~alina/zasoby/data/pl_spacy_model_morfeusz-0.1.3.tar.gz

--2021-01-25 12:00:46--  http://mozart.ipipan.waw.pl/~alina/zasoby/data/pl_spacy_model_morfeusz-0.1.3.tar.gz
Resolving mozart.ipipan.waw.pl (mozart.ipipan.waw.pl)... 213.135.36.148
Connecting to mozart.ipipan.waw.pl (mozart.ipipan.waw.pl)|213.135.36.148|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229068993 (218M) [application/x-gzip]
Saving to: ‘zasoby/pl_spacy_model_morfeusz-0.1.3.tar.gz.1’

pl_spacy_model_morf 100%[===================>] 218.46M  9.84MB/s    in 29s     

2021-01-25 12:01:15 (7.60 MB/s) - ‘zasoby/pl_spacy_model_morfeusz-0.1.3.tar.gz.1’ saved [229068993/229068993]



In [3]:
! python -m pip install zasoby/pl_spacy_model_morfeusz-0.1.3.tar.gz

Processing ./zasoby/pl_spacy_model_morfeusz-0.1.3.tar.gz
  Created wheel for pl-spacy-model-morfeusz: filename=pl_spacy_model_morfeusz-0.1.3-cp36-none-any.whl size=229138616 sha256=77b1bac59405606e944fdfc1779d93475ec60afdb99562298a54a65c8b6e3417
  Stored in directory: /root/.cache/pip/wheels/9c/e8/f2/42317970badff72e337a282a28ad5d714491776d6d8ec1f86d
Successfully built pl-spacy-model-morfeusz
  Found existing installation: pl-spacy-model-morfeusz 0.1.3
    Uninstalling pl-spacy-model-morfeusz-0.1.3:
      Successfully uninstalled pl-spacy-model-morfeusz-0.1.3


In [4]:
!python -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

2021-01-25 12:01:46.182775: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pl_spacy_model_morfeusz -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pl_spacy_model_morfeusz
You can now load the model via spacy.load('pl_spacy_model_morfeusz')


In [5]:
import spacy
nlp = spacy.load("pl_spacy_model_morfeusz")

## Przykładowe zadanie ekstrakcji informacji

### Definicja zadania

**Ile wynoszą dywidendy za akcje spółek notowanych na GPW?**


Pola:

*   nazwa spółki,
*   kwota za akcję.



In [6]:
titles = [
  "Apator wypłaci 1,20 zł dywidendy i skupi akcje własne za 20 mln zł",
  "Allegro SA chce wypłacić 4,08 zł dywidendy na akcję za '18",
  "APS Energia chce wypłacić 4 gr dywidendy na akcję",
  "ATM SA wypłaci 2,46 zł dywidendy na akcję"
]

### Analiza danych

In [7]:
from spacy import displacy

docs = [nlp(title) for title in titles]

for doc in docs:
  if len(doc.ents) > 0:
    displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})
  else:
    print(f"Brak jednostek nazewniczych w: {doc}")


Brak jednostek nazewniczych w: APS Energia chce wypłacić 4 gr dywidendy na akcję


### Wzorce sekwencyjne

In [8]:
from spacy.pipeline import EntityRuler

money_pattern = [{"TEXT": {"REGEX": "^[\d]+([,.][\d]{2})?$"}},
           {"TEXT": {"REGEX": "(zł|PLN|EUR|USD|BGN|gr|grosze)"}}]

company_pattern = [{"LOWER": "aps"}, {"LOWER": "energia"}]

patterns = [{"label": "money", "pattern": money_pattern},
            {"label": "orgName", "pattern": company_pattern}]

nlp2 = spacy.load("pl_spacy_model_morfeusz")

ruler = EntityRuler(nlp2)
ruler.add_patterns(patterns)
nlp2.add_pipe(ruler)

In [9]:
docs2 = [nlp2(title) for title in titles]

for doc in docs2:
  displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

In [10]:
from spacy.matcher import Matcher

matcher = Matcher(nlp2.vocab)
matcher.add("nazwa", None, [{"ENT_TYPE": "orgName"}])

for doc in docs2:
  print(doc)
  matches = matcher(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    print(start, end, span.text)
  print("")

Apator wypłaci 1,20 zł dywidendy i skupi akcje własne za 20 mln zł
0 1 Apator

Allegro SA chce wypłacić 4,08 zł dywidendy na akcję za '18
0 1 Allegro
1 2 SA

APS Energia chce wypłacić 4 gr dywidendy na akcję
0 1 APS
1 2 Energia

ATM SA wypłaci 2,46 zł dywidendy na akcję
0 1 ATM
1 2 SA



In [12]:
for doc in docs2:
  with doc.retokenize() as retokenizer:
    for ent in doc.ents:
      retokenizer.merge(doc[ent.start:ent.end])  

for doc in docs2:
  displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

In [13]:
for doc in docs2:
  print(doc)
  matches = matcher(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    print(start, end, span.text)
  print("")    

Apator wypłaci 1,20 zł dywidendy i skupi akcje własne za 20 mln zł
0 1 Apator

Allegro SA chce wypłacić 4,08 zł dywidendy na akcję za '18
0 1 Allegro SA

APS Energia chce wypłacić 4 gr dywidendy na akcję
0 1 APS Energia

ATM SA wypłaci 2,46 zł dywidendy na akcję
0 1 ATM SA



In [15]:
pattern_dywidenda = [{"ENT_TYPE": "orgName"}, {"TEXT": "chce", "OP": "?"}, 
                     {"LEMMA": "wypłacić"}, {"ENT_TYPE": "money"}]

matcher = Matcher(nlp2.vocab)
matcher.add("dywidenda", None, pattern_dywidenda)                

def process(doc):
  matches = matcher(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    company = [e for e in doc.ents if e.label_ == "orgName"]
    money = [e for e in doc.ents if e.label_ == "money"]
    print(start, end, span.text)
    print((company, money))
  print("")


for doc in docs2:
  print(doc)
  process(doc)

Apator wypłaci 1,20 zł dywidendy i skupi akcje własne za 20 mln zł
0 3 Apator wypłaci 1,20 zł
([Apator], [1,20 zł])

Allegro SA chce wypłacić 4,08 zł dywidendy na akcję za '18
0 4 Allegro SA chce wypłacić 4,08 zł
([Allegro SA], [4,08 zł])

APS Energia chce wypłacić 4 gr dywidendy na akcję
0 4 APS Energia chce wypłacić 4 gr
([APS Energia], [4 gr])

ATM SA wypłaci 2,46 zł dywidendy na akcję
0 3 ATM SA wypłaci 2,46 zł
([ATM SA], [2,46 zł])



### Wykorzystanie parsera zależnościowego

In [16]:
texts_complex = [
  "Amica SA wypłaci 4 zł dywidendy",
  "Amica SA ostatecznie wypłaci 4 zł dywidendy",
  "Amica SA wypłaci w drugiej połowie 2020 roku 4 zł dywidendy",
  "Amica SA, w porozumieniu z akcjonariuszami, wypłaci 4 zł dywidendy",
]

docs_complex = [nlp2(text) for text in texts_complex]

for doc in docs_complex:
  with doc.retokenize() as retokenizer:
    for ent in doc.ents:
      retokenizer.merge(doc[ent.start:ent.end])   

for doc in docs_complex:
  displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

In [17]:
for doc in docs_complex:
  displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

In [18]:
def process_tree(doc):
  
  for token in doc:
    if token.lemma_ == "wypłacić":
      company = None
      money = None
      for child in token.children:
        if child.dep_ == "nsubj" and child.ent_type_ == "orgName":
          company = child
        elif child.dep_ == "obj" and child.ent_type_ == "money":
          money = child

      if company and money:
        print((company, money))


for doc in docs_complex:
  displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})
  process_tree(doc)

(Amica SA, 4 zł)


(Amica SA, 4 zł)


(Amica SA, 4 zł)


(Amica SA, 4 zł)


### Bardziej złożone struktury

In [19]:
texts_even_more_complex = [
    "Amica SA planuje wypłacić 4 zł dywidendy",
    "Amica SA ogłosiła wypłatę 4 zł dywidendy",
    "Amica SA zaakceptowała propozycję wypłaty dywidendy w wysokości 4 zł za akcję",
]

docs_even_more_complex = [nlp2(text) for text in texts_even_more_complex]

for doc in docs_even_more_complex:
  with doc.retokenize() as retokenizer:
    for ent in doc.ents:
      retokenizer.merge(doc[ent.start:ent.end])  

for doc in docs_even_more_complex:
  displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})